# First object condensation model training

* **Requirements*: You need to have graphs constructed, e.g., with `010_build_graphs.ipynb`

In [1]:
from pathlib import Path

from gnn_tracking.postprocessing.dbscanscanner import dbscan_scan

from gnn_tracking.models.track_condensation_networks import GraphTCN
from gnn_tracking.training.tcn_trainer import TCNTrainer
from gnn_tracking.metrics.losses import (
    EdgeWeightFocalLoss,
    PotentialLoss,
    BackgroundLoss,
)
from gnn_tracking.utils.loading import get_loaders, TrackingDataset

In [2]:
graph_dir = Path.home() / "data" / "gnn_tracking" / "graphs"
assert graph_dir.is_dir()

In [9]:
datasets = {
    "train": TrackingDataset(graph_dir, stop=9),
    "val": TrackingDataset(graph_dir, start=9, stop=10),
}
loaders = get_loaders(datasets)

[18:54:32] INFO: DataLoader will load 9 graphs (out of 10 available).
[18:54:32] DEBUG: First graph is /Users/fuchur/data/gnn_tracking/graphs/data21000_s5.pt, last graph is /Users/fuchur/data/gnn_tracking/graphs/data21008_s10.pt
[18:54:32] INFO: DataLoader will load 1 graphs (out of 10 available).
[18:54:32] DEBUG: First graph is /Users/fuchur/data/gnn_tracking/graphs/data21008_s24.pt, last graph is /Users/fuchur/data/gnn_tracking/graphs/data21008_s24.pt
[18:54:32] DEBUG: Parameters for data loader 'train': {'batch_size': 1, 'num_workers': 1, 'shuffle': True, 'pin_memory': True}
[18:54:32] DEBUG: Parameters for data loader 'val': {'batch_size': 1, 'num_workers': 1, 'shuffle': False, 'pin_memory': True}


In [4]:
loss_functions = {
    "edge": EdgeWeightFocalLoss(gamma=5, alpha=0.95),
    "potential": PotentialLoss(q_min=0.01),
    "background": BackgroundLoss(sb=1),
}

loss_weights = {
    "edge": 500,
    "potential_attractive": 500,
    "potential_repulsive": 5,
    "background": 0.05,
}

In [10]:
model = GraphTCN(
    node_indim=datasets["train"].num_node_features,
    edge_indim=datasets["train"].num_edge_features,
    h_dim=10,
    e_dim=10,
    L_ec=5,
    L_hc=2,
    h_outdim=10,
    hidden_dim=128,
)

In [6]:
clustering_functions = {"dbscan": dbscan_scan}

In [11]:
trainer = TCNTrainer(
    model=model,
    loaders=loaders,
    loss_functions=loss_functions,
    lr=0.005,
    loss_weights=loss_weights,
    cluster_functions=clustering_functions,
)

[18:55:04 TCNTrainer] INFO: Using device cpu


In [12]:
trainer.train(epochs=2)

[18:55:06 TCNTrainer] INFO: Epoch  1 (    0/9): edge_weighted=   0.83609, potential_attractive_weighted=   0.00020, potential_repulsive_weighted=2477.93213, background_weighted=   0.05000
[18:55:08 TCNTrainer] INFO: Training for epoch 1 took 3.34 seconds
/Users/fuchur/.miniconda3/envs/gnn/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/fuchur/.miniconda3/envs/gnn/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3747: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/fuchur/.miniconda3/envs/gnn/lib/python3.9/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
[18:55:09 ClusterHP] WARNING: Only using 1 thread for DBSCAN scan
[18:55:09 ClusterHP] DEBUG: Starting from params: {'eps': 0.95, 'min_